# Kubernetes Version (Current)

In [20]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

from ipynb.fs.full.kubernetesTool import *

from ipynb.fs.full.invoiceSmartcontract import *

## BaseAction

In [41]:
class BaseAction(object):
    
    def __init__(self):
        self.name = __class__.__name__
        self.state = None
    
    def dict(self):
        if self.state is None:
            return {'class' : self.__class__.__name__, 'parameters' : self.__dict__}
        return {'class' : self.__class__.__name__, 'parameters' : self.state }
    
    def merged_dict(self,parameters=None):
        old = dict()
        return {'class':old['class'],'parameters':{**old['parameters'],**parameters}}
    
    def merge(self,parameters={}):
        adict = {'class' : self.__class__.__name__, 'parameters' : {**self.__dict__, **parameters}}
        #adict = merged_dict(parameters)
        return eval(adict['class'])(**adict['parameters'])
    
    def _get_image(self,image):
        if isinstance(image, dict):
            return image['image']
        return image
    
    def _filename(self,filename=None,name=None):
        if name != None:
            return name + '_' + self.__class__.__name__ + '.json'
        if filename == None:
            return '_action_' + self.__class__.__name__ + '.json'
        return filename
    
    def save(self,filename=None,name=None):
        with open(self._filename(filename,name), 'w', encoding='utf-8') as f:
            json.dump(self.dict(), f, ensure_ascii=False, indent=4)    

    def load(self,filename=None,name=None):
        with open(self._filename(filename,name), 'r') as fp:
            file = json.load(fp)  
            return file          

### SharePoint

In [22]:
import subprocess

class SharePoint(object):
    def __init__(self,host='10.15.130.111:80',user='scray', pwd='scray'):
        self.host = host
        self.user = user
        self.pwd  = pwd
    
    def downloadToLocal(self, local_path='./', local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, 
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)     

    def deleteRemote(self, local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, '-X', 'DELETE',
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)    

    def createDirectoryRemote(self, local_name='' ,remote_path='upload'):  
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, '-X', 'MKCOL',
                                                   'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)       

    def uploadFromLocal(self, local_path='./', local_name='' ,remote_path='upload'):
        try:    
            output = subprocess.check_output(['curl', '--user',self.user + ':' + self.pwd, 
                                              '-T', local_path + '/' + local_name, 
                                              'http://' + self.host + '/' + remote_path + '/' + local_name])
            return output.decode('ascii')
        except Exception as e:
            return str(e)   

    def downloadToLocalCmdList(self, local_path='./', remote_name='' ,remote_path='/'):  
        return ['curl', '--user',self.user + ':' + self.pwd, 'http://' + self.host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]    

## KubernetesPeer

In [36]:
#import kubernetesTool
import subprocess

class KubernetesPeer(BaseAction): 
    def __init__(self, name='',prefix='peer', hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        super(KubernetesPeer, self).__init__()
        self.name = name
        
        ## for dict
        self.state={}
        self.state['name']=name
        self.state['hostname']=hostname
        self.state['ca_country']=ca_country
        self.state['ca_province']=ca_province
        self.state['ca_locality']=ca_locality
        
        #self.pod_name = kubernetesTool.getPod(name)
        self.pod_name = getPod(name)
        if self.pod_name is None:
            self.addPeer(hostname,ca_country,ca_province,ca_locality)
            self.pod_name = getPod(name)
        
        #self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        #self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        self.listen_port = getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = getPort(peername=name, name = prefix + '-gossip')
        
        self.hostname = self.get_hostname()
        self.wallet_tool = None
        self.organizations = {}
        #self.smart_contract = None
        
        
    def get_hostname(self):
        return executePeerCmd(self.pod_name,'echo $HOSTNAME')[:-1]
            
    def addPeer(self, hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten', prefix='peer'):  
        peername = self.name
        #if peername in peers.keys():
        #    return peers[peername]
        # Create configuration for new peer
        executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
        # Start service
        executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
        #elf.listen_port = kubernetesTool.getPort(peername=peername, name = prefix + '-listen')
        #self.gossip_port = kubernetesTool.getPort(peername=peername, name = prefix + '-gossip')
        self.listen_port = getPort(peername=peername, name = prefix + '-listen')
        self.gossip_port = getPort(peername=peername, name = prefix + '-gossip')
        
        # Create peer configuration
        res = self.createConfig(hostname=hostname,
                                ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        # Start new peer
        executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
        #peer = kubernetespeer(name=peername)
        #peers[peername] = self
        #return peer        
        
    def addPeerFromKubernetes():
        peer= kubernetespeer(self.name)
        peers[self.name] = peer
        
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)    
        
    def createConfig(self,hostname='kubernetes.research.dev.seeburger.de' ,ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
        try:  
            peername=self.name
            peer_listen_port=self.listen_port
            #peer_gossip_port=self.gossip_port
            peer_gossip_port=self.listen_port
            address = 'peer0.' + peername  + '.' +  hostname + ':'
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                              '--from-literal=hostname=' + peername  + '.' + hostname,
                                              '--from-literal=org_name=' + peername,
                                              '--from-literal=data_share=hl-fabric-data-share-service:80',
                                              '--from-literal=ca_country='  + ca_country,
                                              '--from-literal=ca_province=' + ca_province,
                                              '--from-literal=ca_locality=' + ca_locality,
                                              '--from-literal=CORE_PEER_ADDRESS=' + address + peer_listen_port,
                                              '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=' + address + peer_gossip_port,
                                              '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
            return str(output)
        except Exception as e:
            return str(e)  

    
    def deleteConfig(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + self.name])
            return str(output)
        except Exception as e:
            return str(e)     
        
    def deleteDeployment(self):
        try:  
            output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','deployment', self.name])
            return str(output)
        except Exception as e:
            return str(e)           

    def deletePeer(self):
        self.deleteDeployment()
        self.deleteConfig()        
        
    def joinHyperledger(self,orderer_ip=None,orderer_hostname=None,orderer_port=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):          
        cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
        print(cmd)
        return executePeerCmd(self.pod_name, cmd)

    def endorse_peer(self,orderer=None,orderer_ip=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):
        peername = self.name
        peer_host_name = self.get_hostname()
        #peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
        print(orderer_ip,channel_name,self.name,shared_fs_host,ext_peer_ip, peer_host_name)
        cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,self.name,shared_fs_host,ext_peer_ip, peer_host_name]) 
        #print(cmd)
        return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

    def endorse_and_join(self,orderer=None,orderer_ip=None,channel_name=None,shared_fs_host=None,ext_peer_ip='10.15.136.41'):    
        res1 = self.endorse_peer(orderer=orderer,orderer_ip=orderer_ip,channel_name=channel_name,shared_fs_host=shared_fs_host)   
        res2  = self.joinHyperledger(orderer_ip=orderer_ip,orderer_hostname=orderer.host,orderer_port=orderer.listen_port,channel_name=channel_name,shared_fs_host=shared_fs_host,ext_peer_ip=ext_peer_ip)
        return res1,res2        
        
    def installAndAproveChaincode(self, orderer=None, channel_name=None, chain_code=None, sequence=1):
        strlist = ['/mnt/conf/install_and_approve_cc.sh', chain_code.host, orderer.get_ip(), orderer.host, str(orderer.listen_port), channel_name, chain_code.pkgid, chain_code.hostname, chain_code.label, chain_code.shared_fs, str(sequence) ]
        return executePeerCmd(self.pod_name,toCmd(strlist)) 

    def commitChaincode(self,channel_name,sequence=1,pkgid=''):
        strlist = ['/mnt/conf/peer/cc_commit.sh', channel_name, str(sequence),pkgid ]
        print(toCmd(strlist))
        return executePeerCmd(self.pod_name,toCmd(strlist))                 
        
       
    #def getExport_MSPCONFIGPATH(self,hostname,user='User1'):
    def getExport_MSPCONFIGPATH(self,user='User1'):
        splits = user.split('@')
        hostname =  self.hostname 
        if len(splits) > 1:
            user = splits[0]
            hostname = splits[1]
        path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
        return 'export CORE_PEER_MSPCONFIGPATH=' + path
        
    def invoke(self,callopt,channel_name=None, user='User1'):
        #print(callopt)
        global _out
        _out = widgets.Output()
        with _out:
            try:
                #export_cmd = self.getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
                export_cmd = self.getExport_MSPCONFIGPATH(user=user)
                cmd0 = str('\'' + callopt + "\'")
                #strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
                #strlist = ['peer', 'chaincode', 'invoke', '--waitForEventTimeout','1s' ,'-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
                strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--waitForEvent',  '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]

                cmd = toCmd(strlist)
                cmd = export_cmd + '  && ' + cmd
                #print(cmd)  
                output = executePeerCmd(self.pod_name, cmd)
                if output.endswith ('\n'):
                    return output[:-1],_out
                return output,_out
            except Exception as e:
                print('Exception:', e)
                if output.endswith ('\n'):
                    return output[:-1],_out
                return output,_out

    def query(self,callopt, channel_name=None, user='User1'):
        global _out
        _out = widgets.Output()
        with _out:
            try:
                #export_cmd = self.getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
                export_cmd = self.getExport_MSPCONFIGPATH(user=user)
                cmd0 = str('\'' + callopt + "\'")
                strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
                cmd = toCmd(strlist)
                cmd = export_cmd + '  && ' + cmd
                print(cmd)
                output = executePeerCmd(self.pod_name, cmd)
                if output.endswith ('\n'):
                    output=output[:-1]
                return json.loads(output),_out
            except Exception as e:
                print('Exception:', e)
                return output,_out        
        
    def installWalletTool(self,sharepoint=None,channel_name=None):
        self.wallet_tool = WalletTool(peer=self,sharepoint=sharepoint)
        self.wallet_tool.installWalletTool()        
        
        organizations = self.wallet_tool.getPeerOrganizations()
        for _organization in organizations:
            porg = Organization(name=_organization)
            _users = self.wallet_tool.getPeerUsers(_organization)
            for _user in _users:
                puser = User(peer=self, name=_user, channel_name=channel_name)
                porg.addUser(user=puser)
            self.organizations[_organization]=porg
        
        
#class ordererpeer(kubernetes_peer):
class ordererpeer(object):
    def __init__(self, name='',prefix='orderer'):
        #super().__init__(name,prefix)
        self.name = ''
        #self.pod_name = kubernetesTool.getPod(name)
        self.pod_name = getPod(name)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        #self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.listen_port = getPort(peername=name, name = prefix + '-listen')
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [39]:
KubernetesPeer(name='aubonmoulin', hostname='fr',ca_country='FR',ca_province='Provence-Alpes-Cote d\'Azur',ca_locality='Malaucene').dict()

{'class': 'KubernetesPeer',
 'parameters': {'name': 'aubonmoulin',
  'hostname': 'fr',
  'ca_country': 'FR',
  'ca_province': "Provence-Alpes-Cote d'Azur",
  'ca_locality': 'Malaucene'}}

## Blockchain

In [24]:
class Blockchain(BaseAction):
    def __init__(self, name='orderer-org1-scray-org',prefix='orderer', channel_name='invoicing17',shared_fs=None,sharepoint=None):
        super(Blockchain, self).__init__()
        self.peers = {}
        
        self.orderer = ordererpeer(name=name,prefix='orderer')
        self.orderer_ip=self.orderer.get_ip()
        self.orderer_hostname=self.orderer.host
        self.orderer_port= self.orderer.listen_port

        self.shared_fs = shared_fs
        self.channel_name=channel_name
        #self.shared_fs_host='10.14.128.38:30080'
        self.sharepoint = sharepoint
        self.wallet_tool = WalletTool(sharepoint=self.sharepoint)
 
        self.installWalletTool()
        
        
    
    def addPeer(self,name=None,hostname='kubernetes.research.dev.seeburger.de',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):  
        if name in self.peers.keys():
            return self.peers[name]

        peer = KubernetesPeer(name=name,hostname=hostname,ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
        self.peers[name] = peer   

    def add_channel(self,name):
        self.channel_name=name
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', self.orderer.pod_name, '-c', 'scray-orderer-cli','--', '/bin/sh']
        cmd2 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '30081']
        #cmd = toCmd(cmd1 + cmd2)
        ret = subprocess.run(cmd1 + cmd2, stdout = subprocess.PIPE, stderr=subprocess.PIPE)
        #return executePeerCmd(orderer.pod_name, cmd, cli='scray-orderer-cli')
        return ret.stdout.decode('ascii'), ret.stderr.decode('ascii')    
    
    def endorse_and_join_all_peers(self): 
        out2 = widgets.Output()
        with out2:
            for key, peer in self.peers.items():
                peer.endorse_and_join(orderer=self.orderer,orderer_ip=self.orderer_ip,channel_name=self.channel_name,shared_fs_host=self.shared_fs)
        return out2      
    
    def deletePeers(self):
        for key,peer in self.peers.items():
            peer.deletePeer()
        for key in list(self.peers.keys()):    
            del self.peers[key]

    # create, delete numbered peers
    def createPeers(self,base_name,start=0, end=10):
        for x in range(start,end):
            peername = base_name + '-' + str(x) 
            self.addPeer(peername)

    def deleteNumberedPeers(name):
        for x in range(0,10):
            key = name + str(x)
            self.peers[key].deletePeer()    
            del self.peers[key]
           
    # kubernetes    
    def deleteAllPeers(config):
        for item in config['items']:
            try:
                if 'org_name' in item['data'].keys():
                    self.deletePeer(item['data']['org_name'])
            except Exception as e:
                print(item['data'])    
         
    def installAndAproveChaincode(self, chain_code=None):       
        for key, peer in self.peers.items():
            peer.installAndAproveChaincode(orderer=self.orderer, channel_name=self.channel_name, chain_code=chain_code)   
            
    def installWalletTool(self):
        for key,peer in self.peers.items():
            peer.installWalletTool(sharepoint=self.sharepoint,channel_name=self.channel_name)
            #wallet_tool = WalletTool(peer=peer,sharepoint=self.sharepoint)
            #wallet_tool.installWalletTool()
            
           
            
    def getUser(self, _user):
        _org = _user.split('@')[1]
        for key,peer in self.peers.items():
            if _org in peer.organizations.keys():
                if _user in peer.organizations[_org].users.keys():
                    return peer.organizations[_org].users[_user]    
        return None

## Chaincode

In [25]:
import json

class Chaincode(BaseAction):
    def __init__(self,hostname='invoice-net.org1.example.com',service_name='hl-fabric-cc-external-invoice-net'):
        super(Chaincode, self).__init__()
        #self.shared_fs='kubernetes.research.dev.seeburger.de:30080'
        self.shared_fs='10.15.130.111:80'
        self.hostname=hostname
        self.host='10.14.128.38'
        self.service_name=service_name
        self.port = executeKubectlCmd(['get', 'service', self.service_name, '-o', 'jsonpath="{.spec.ports[?(@.name==\'chaincode\')].nodePort}"'], decode='json')
        self.label='basic_1.0'
        self.pkgid = self.get_pkgid()
        
    def get_pkgid(self):
        try:
            strlist = ['curl','-s','--user','scray:scray','http://' + self.shared_fs + '/cc_descriptions/' + self.hostname + '_' + self.label + '/description-hash.json']
            output = executeLocalCmd(strlist)
            return json.loads(output.decode('ascii'))['description-hash']
        except Exception as e:
            return None   

## User

In [40]:
class User(BaseAction):
    def __init__(self,peer=None, name=None, channel_name=None):
        super(User, self).__init__()
        self.peer=peer
        self.name=name
        self.smart_contract = InvoiceSmartContract(peer=self.peer, user=self.name, channel_name=channel_name)
        
        self.state={}
        self.state['parameter_peer'] = self.peer.name
        self.state['name']           = self.name

## Organization

In [43]:
class Organization(BaseAction):
    def __init__(self,name=None):
        super(Organization, self).__init__()
        self.name=name
        self.users={}
        
    def addUser(self,user=None):
        self.users[user.name]=user

In [44]:
Organization().dict()

{'class': 'Organization',
 'parameters': {'name': None, 'state': None, 'users': {}}}

# Block

In [45]:
import base64

class Block(BaseAction):
    def __init__(self,peer=None,channel_name=None,sharepoint=None):
        super(Block, self).__init__()
        self.channel_name = channel_name
        self.sharepoint = sharepoint
        self.peer = peer
    
    def get_height(self):
        global _out
        _out = widgets.Output()
        with _out:
            strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', self.channel_name]
            cmd = toCmd(strlist)
            output = executePeerCmd(self.peer.pod_name,  cmd)
            return int(json.loads(output)['height'])

    def uploadBlockToShare(self,id=None):
        global _out
        _out = widgets.Output()
        with _out:        
        
            if id is None:
                id = self.get_height() -1

            strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', self.channel_name, '--block', str(id),'--publish', self.sharepoint.host]
            cmd = toCmd(strlist)
            output = executePeerCmd(self.peer.pod_name,  cmd)

    def getBlockFromShare(self,id=None):
        global _out
        _out = widgets.Output()
        with _out:
            if id is None:
                id = self.get_height() -1

            # invoicing25-block-63.block.json    
            filename =  self.channel_name + '-block-' + str(id) + '.block.json'
            print(filename)
            downloadToLocal(host=self.sharepoint.host,user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/' + self.channel_name)
            output = subprocess.check_output(['cat', '/tmp/' + filename])
            return json.loads(output.decode('ascii'))           

    def decode_block_data(self,block=None):
        _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
        if _transaction['value'] != None:
            return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))

    def decode_block_transaction(self,block=None):
        _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
        result = []
        for arg in _transaction:
            result.append(base64.b64decode(arg).decode("utf-8"))
        return result        

## wallet tool

In [ ]:
import ipywidgets as widgets

class WalletTool(BaseAction):
    def __init__(self,peer=None,sharepoint=None):
        super(WalletTool, self).__init__()
        self.peer = peer
        self.sharepoint = sharepoint
    
    def uploadWalletToolToSharepoint(self):
        self.sharepoint.uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
        self.sharepoint.uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

    def installPeerWalletTool(self):
        executePeerCmd(self.peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
        #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
        executePeerCmd(self.peer.pod_name,'apk add openssl')
        executePeerCmd(self.peer.pod_name,'apk add openjdk8-jre')
        # chmod 755

    def createPeerUser(self,user=None, admin='admin',
        cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
        key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
            cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
                   '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
            return executePeerCmd(self.peer.pod_name,toCmd(cmd))

    def installWalletTool(self):
        _out = widgets.Output()
        with _out:
            self.installPeerWalletTool()
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name,  cmd)
            executePeerCmd(self.peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name,  cmd)
            executePeerCmd(self.peer.pod_name,'chmod 755 /tmp/create-user.sh')  
        return _out   

    def addUser(self,user,org,org2):
        _out = widgets.Output()
        with _out:
            strlist = ['/tmp/create-user.sh', user,org,org2,'10.15.130.111:80']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(self.peer.pod_name, 'cd /tmp && ' + cmd)    
        return output,_out      
    
    def getPeerUsers(self,org):
        strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
        #print(toCmd(strlist))
        users=[]
        lines=executePeerCmd(self.peer.pod_name,toCmd(strlist)).splitlines() 
        for line in lines:
            #users.append(line.split('@')[0])
            users.append(line)
        return users

    def getPeerOrganizations(self):
        strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
        return executePeerCmd(self.peer.pod_name,toCmd(strlist)).splitlines() 

    def get_peer_of_user(self,user=None,peers=None):
        for key,peer in peers.items():
            orgs=getPeerOrganizations(peer)
            for org in orgs:
                #print(peer.pod_name, org)
                users = getPeerUsers(peer,org)
                for ouser in users:
                    if user == ouser:
                        return peer    